# Index of `planning/`

Given the number of known unknowns in this project (at least to myself), it's reasonable to start out with a bit of in-depth research and planning.  This directory is devoted to that work.  Each notebook is focused on a set of closely-related questions, and proceed in a bit of an order.

As a reminder of the components of this system: we have a camera, connected to a telescope, which can scan across the sky, following a satellite whose position is predicted from TLEs.

A key question to answer is "Where is the satellite right now?"  The mechanisms behind this are a huge blind spot for myself, so it's where I dug in first.

## Obsolete-ish things
[(Obsolete) Simple ECI Frames](simple_eci_frames.ipynb) is the first thing I did here. There's a lot of work in figuring out how to model Earth-Centered Inertial frames.  These allow us to find where two objects on or around the Earth are in the same reference frame.  With that, we can find the vector between the two, and convert that into an alt,az tuple for the ground observer.  This part took a couple days to see through, as it involved a lot of delving through poorly-documented C code from the 80s (yes, really).  It all turned out to be moot for satellites, though, as PyEphem supports this kind of measurement right out of the box.  However, the code did turn out to be moderately handy when I sidetracked into ADS-B (I need something moving relatively quickly to track when it's cloudy out…).

[(Obsolete) Rendezvous with Zarya](obsolete_rendezvous_with_zarya.ipynb) is an attempt to use the ECI frames work to track the ISS.  However, partway through, I tried using PyEphem and found it was super handy, so I aborted this.  Before that, I tried two other python ephemerides calculators, both of which were more or less terrible.  PyEphem was great, though, so I could throw away both this file and the last one.

## First parameters computed

[Passes to Parameters](passes_to_parameters.ipynb): now that we can predict satellite passes, it's time to measure the properties the system will need to have in order to succeed.  This notebook pulls in a week's worth of all satellite passes at my house, then measures the speed at which our telescope needs to slew around the sky.  This also provided a good opportunity for a first stab at optical limits, to see what our system could see, as it were.

### Results:

In order to track most of the targets, the telescope needs to be able to:

* slew at about 0.2°/second in azimuth
* slew at about 0.2°/second in altitude (weird coincidence)

## Converting those speeds into drivetrain specs

[Drive train parameters](drive_train_parameters.ipynb) uses the tracking parameter from the previous step and adds in knowledge about the optical system to figure out the needed tracking precision and accuracy.

It turns out that pixel-perfect tracking is hilariously impossible, which only makes sense when each pixel is an arcsecond wide.  Given a 200 step/revolution stepper (standard 1.8°/step), we'd need a 6,480:1 gear ratio for our 1'' pixels.  And, if we used the microscope webcam at UXGA mode, it's 0.62''/pixel, yielding an amazing 10,472:1 geardown.  That's implausible, so we start looking at it for the simpler "keep the object within one third of the frame" case.  In the UXGA webcam case, that's still a 20:1 ratio, and is probably only usable for the brightest of objects (I haven't actually measured the limiting magnitude for that configuration yet).

In the end, we need a 1,000:1 gearing ratio to get down to 6.5''/step, which is roughly on the order of the seeing we can expect to get.  If we're using the 500D in 720p video mode, each pixel is 3.6'', so that's a reasonable configuration.  This also presumes the focusing is usable, which it really hasn't been in astrophotography on the 127EQ.

In any case, this left me with a bit of a pickle: getting this level of resolution is available off-the-shelf, but it's far from cheap.  It's not clear that this project is worth that expense at the moment, so I need to continue noodling on ways to meet these needs without breaking the budget.

## IMU Experimentation

In [the imu/ directory](../imu/) (under the top level of this project), you'll find some Arduino and python code to try and gather data from an IMU.  The idea was to see if I could close the loop with a magnetometer and accelerometer (Narrator: he couldn't).  The magnetometer readings were noisy, coarse, and very sensitive to anything in the vicinity disturbing the magnetic field.  This includes screwdrivers, appliances, and lord help you if you run a stepper around it.  Given that the intensity of the local magnetic field is roughly 40µT, it makes a bit more sense: that's 1% of a refridgerator magnet.

I did get both the magnetometer and the accelerometer working on the same I2C bus, though, and it may be worth revisiting this for rough headings.  It may also be worth considering something moderately awful like putting a ramp around the outside of the telescope azimuth stage, then leaning one end of a fixed accelerometer to it. (It's probably better to just put an encoder strip around the outside, though.)

In any case, this was a total bust in closing the loop, so we still need something.  Using astrometry.net for Push-To operation worked pretty reasonably the other night, so this may be useful for getting rough calibration.  From there, the scope will need to either hunt for the target (or use a smaller ride-along camera with a wider FOV to pre-acquire the target).

## What can we do without a telescope?

[Upcoming birbs](upcoming_passes_in_window.ipynb): it occurred to me that a static camera might be able to capture interesting things without the scope.  Brighter satellites are probably visible to it, so maybe it's worth trying?  The camera can only see so much of the sky (measured at 20°×16°), so where should it be pointed during those windows?  To find out, we simulate a bunch of passes during the relevant time windows, then smooth them with a 20°×16° window.  This gets us some pretty sunrise/sunset images of where to point a camera to possibly see satellites.

I tried this one night, but the mirror slap of the camera woke me up shortly after it started shooting (at 4:30 in the morning…), so I canceled this attempt.  I may try this again, though, as it seems promising.  Just in the evening configuration, instead of the morning one.

## What about airplanes?

That got me thinking, though: there are lots of planes in the same direction I was pointing for satellites.  What if we started tracking them?  This also gives us something to practice tracking on the proverbial cloudy nights, as the airplanes are (generally) _under_ the clouds, so they're visible even when the sky isn't!

Instead of waffling about with possible-or-not, I decided to just cut some code for this.  The net result is found in [adsb_test/adsb_hud.py](ADS-B HUD), which works great right up to the point where planes are invisible because the webcam I'm using is a piece of junk. `¯\_(ツ)_/¯`  Turns out that theory's good for something, kids!  It creates an mjpeg stream you can view in a browser, where circles are drawn on the all-black image of the sky, tracking the planes that are there (but invisible because crap webcam).  Overall, it was fun to write, and I'm going to improve on it a bit by getting a camera that isn't crap, as it's a fun bit of goofiness (it's also tempting to port to an iPhone AR app or some such).